In [2]:
frames_path = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\unzipped_video"


In [3]:
!pip show opencv-python tqdm matplotlib


Name: opencv-python
Version: 4.11.0.86
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/opencv/opencv-python
Author: 
Author-email: 
License: Apache 2.0
Location: C:\Users\simar\anaconda3\Lib\site-packages
Requires: numpy, numpy, numpy, numpy, numpy
Required-by: 
---
Name: tqdm
Version: 4.65.0
Summary: Fast, Extensible Progress Meter
Home-page: https://tqdm.github.io
Author: 
Author-email: 
License: MPLv2.0, MIT Licences
Location: C:\Users\simar\anaconda3\Lib\site-packages
Requires: colorama
Required-by: anaconda-client, anaconda-project, conda, conda-build, gdown, nltk, openai, panel
---
Name: matplotlib
Version: 3.8.0
Summary: Python plotting package
Home-page: https://matplotlib.org
Author: John D. Hunter, Michael Droettboom
Author-email: matplotlib-users@python.org
License: PSF
Location: C:\Users\simar\anaconda3\Lib\site-packages
Requires: contourpy, cycler, fonttools, kiwisolver, numpy, packaging, pillow, pyparsing, python-dateutil
Required-by: se

In [4]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


In [6]:
orb = cv2.ORB_create(nfeatures=1000)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

frames = sorted(os.listdir(frames_path))
frame_paths = [os.path.join(frames_path, f) for f in frames]
affine_transforms = []

for i in range(len(frame_paths) - 1):
    img1 = cv2.imread(frame_paths[i], cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(frame_paths[i + 1], cv2.IMREAD_GRAYSCALE)
    if img1 is None or img2 is None:
        continue
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    if des1 is None or des2 is None or len(kp1) < 4 or len(kp2) < 4:
        continue
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    if len(pts1) < 4:
        continue
    affine, _ = cv2.estimateAffinePartial2D(pts1, pts2)
    if affine is not None:
        affine_transforms.append(affine)

In [5]:
class KalmanFilter1D:
    def __init__(self, process_variance=1e-3, measurement_variance=1e-1):
        self.process_variance = process_variance
        self.measurement_variance = measurement_variance
        self.posteri_estimate = 0
        self.posteri_error_estimate = 1.0
    def filter(self, measurements):
        estimates = []
        for measurement in measurements:
            prior = self.posteri_estimate
            prior_err = self.posteri_error_estimate + self.process_variance
            K = prior_err / (prior_err + self.measurement_variance)
            self.posteri_estimate = prior + K * (measurement - prior)
            self.posteri_error_estimate = (1 - K) * prior_err
            estimates.append(self.posteri_estimate)
        return np.array(estimates)

dx = [t[0, 2] for t in affine_transforms]
dy = [t[1, 2] for t in affine_transforms]
kf_x, kf_y = KalmanFilter1D(), KalmanFilter1D()
dx_smooth, dy_smooth = kf_x.filter(dx), kf_y.filter(dy)

smoothed_transforms = []
for i, affine in enumerate(affine_transforms):
    smoothed = affine.copy()
    smoothed[0, 2], smoothed[1, 2] = dx_smooth[i], dy_smooth[i]
    smoothed_transforms.append(smoothed)

In [6]:
video_name= r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\DJI_20250411113208_0019_D.MP4"
cap = cv2.VideoCapture(video_name)
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Video FPS: {fps}")

Video FPS: 29.97002997002997


In [ ]:
sample_frame = cv2.imread(frame_paths[0])
h, w = sample_frame.shape[:2]
out_path = "ORB19VSstabilized_output_full.mp4"
out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

for i in tqdm(range(len(smoothed_transforms))):
    frame = cv2.imread(frame_paths[i])
    if frame is None:
        continue
    M = smoothed_transforms[i]
    stabilized = cv2.warpAffine(frame, M, (w, h))
    out.write(stabilized)

last_frame = cv2.imread(frame_paths[-1])
if last_frame is not None:
    out.write(last_frame)

out.release()
print(" Stabilized video saved as:", out_path)

  0%|          | 0/7168 [00:00<?, ?it/s]

 Stabilized video saved as: 19VSstabilized_output_full.mp4
